# Original Routes Part 2

### Setting Up

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import networkx as nx
import shapely
import folium
import geojson
import math
import osmnx as ox
from rtree import index as rtree_index
import pickle
import copy
import utm
import csv

from shapely.ops import unary_union
from shapely.geometry import Polygon, MultiPolygon, LineString, Point
from geopy.distance import geodesic
from shapely.ops import split

from __future__ import absolute_import, division
from math import radians, sin, cos, sqrt, atan2, exp, log
import webbrowser
import random
from scipy.spatial import KDTree
from scipy.spatial.distance import euclidean

ox.settings.log_console=True
ox.settings.use_cache=True

In [8]:
# Defining classes for the dataframes      
class stopCandidate:
    def __init__(self, lat, long, isTranspo, id):
        self.lat = lat
        self.long = long
        self.isTranspo = isTranspo
        self.id = id #Stop ID
        
    def getLat(self):
        return self.lat
    
    def getLong(self):
        return self.long
    
    def getDegree(self):
        return self.degree
    
class networkObj:
    def __init__(self, route_network, graph):
        self.fitness_score = 0
        self.graph = graph
        self.route_network = route_network
        
class Route:
    def __init__(self, route, route_id, distance):
        self.route = route
        self.route_id = route_id
        self.distance = distance

In [9]:
# For units
def degrees_to_meters(angle_degrees):
    return angle_degrees * 6371000 * math.pi / 180

def meters_to_degrees(distance_meters):
    return distance_meters / 6371000 * 180 / math.pi

In [10]:
# Import and Export networks or graphs to pickle
def export_networks(networks, path):
    with open(path, 'wb') as f:
        pickle.dump(networks, f)

def import_networks(path):
    with open(path, 'rb') as f:
        routes = pickle.load(f)
    return routes

#### Graph and Features

In [11]:
# GENERATION OF MAIN CITY GRAPH
# IF FIRST TIME RUNNING, RUN THIS CODE TO GENERATE THE GRAPH
def generate_graph():
    mode = 'drive'
    graph = ox.graph_from_place(select_city, network_type = mode) # Generate graph of Metro manila
    ox.save_graphml(graph, city_file) # Save it as a file

def load_graph():
    graph = ox.load_graphml(city_file)
    
    print("Graph loaded successfully")
    print("NUMBER OF EDGES: ", graph.number_of_edges())
    print("NUMBER OF NODES: ", graph.number_of_nodes())
    print('\n')
    return graph

# select_city = "Metro Manila, Philippines"
# city_file = 'map/Metro Manila.graphml'
# #generate_graph()
# METROMANILA_GRAPH = load_graph()

select_city = "Manila, Philippines"
city_file = 'map/Manila.graphml'
#generate_graph()
MANILA_GRAPH = load_graph()

select_city = "Mandaluyong, Philippines"
city_file = 'map/Mandaluyong.graphml'
#generate_graph()
MANDALUYONG_GRAPH = load_graph()

select_city = "Makati, Philippines"
city_file = 'map/Makati.graphml'
#generate_graph()
MAKATI_GRAPH = load_graph()

Graph loaded successfully
NUMBER OF EDGES:  12617
NUMBER OF NODES:  4926


Graph loaded successfully
NUMBER OF EDGES:  2374
NUMBER OF NODES:  989


Graph loaded successfully
NUMBER OF EDGES:  5505
NUMBER OF NODES:  2270




In [12]:
### For Filtering the roads and other features
# GETTING ROADS AND WATERWAYS

# Get all the roads in Manila
manila_road = ox.graph_to_gdfs(MANILA_GRAPH,nodes=False, edges=True)
filtered_roads = manila_road[manila_road['junction'].isna()]

# Separate roads whose highway types are only one value and those that are more than 1 (lists)
rows_with_lists = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, list))]
rows_with_strings = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, str))]

# Allowed Roads to place stops
filter_options = ['primary', 'secondary', 'tertiary', 'trunk', 'unclassified']

# Get all the roads with the allowed road types
filtered_roads_strings_manila = rows_with_strings.loc[rows_with_strings['highway'].isin(filter_options)] 

# Total Distance of the allowed road types
TOTAL_NETWORK_ROAD_DISTANCE_MANILA = 0
for index, row in filtered_roads_strings_manila.iterrows():
    TOTAL_NETWORK_ROAD_DISTANCE_MANILA += row['length']
    
# MAKATI
makati_road = ox.graph_to_gdfs(MAKATI_GRAPH,nodes=False, edges=True)
filtered_roads = makati_road[makati_road['junction'].isna()]

# Separate roads whose highway types are only one value and those that are more than 1 (lists)
rows_with_lists = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, list))]
rows_with_strings = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, str))]

# Allowed Roads to place stops
filter_options = ['primary', 'secondary', 'tertiary', 'trunk', 'unclassified']

# Get all the roads with the allowed road types
filtered_roads_strings_makati = rows_with_strings.loc[rows_with_strings['highway'].isin(filter_options)] 

# Total Distance of the allowed road types
TOTAL_NETWORK_ROAD_DISTANCE_MAKATI = 0
for index, row in filtered_roads_strings_makati.iterrows():
    TOTAL_NETWORK_ROAD_DISTANCE_MAKATI += row['length']


# MANDALUYONG
mandaluyong_road = ox.graph_to_gdfs(MANDALUYONG_GRAPH,nodes=False, edges=True)
filtered_roads = mandaluyong_road[mandaluyong_road['junction'].isna()]

# Separate roads whose highway types are only one value and those that are more than 1 (lists)
rows_with_lists = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, list))]
rows_with_strings = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, str))]

# Allowed Roads to place stops
filter_options = ['primary', 'secondary', 'tertiary', 'trunk', 'unclassified']

# Get all the roads with the allowed road types
filtered_roads_strings_mandaluyong = rows_with_strings.loc[rows_with_strings['highway'].isin(filter_options)] 

# Total Distance of the allowed road types
TOTAL_NETWORK_ROAD_DISTANCE_MANDALUYONG = 0
for index, row in filtered_roads_strings_mandaluyong.iterrows():
    TOTAL_NETWORK_ROAD_DISTANCE_MANDALUYONG += row['length']

### Functions

In [13]:
# DELETE

def add_point_to_graph(point, graph_of_stops, list_of_stops, is_transpo):
    lat = point.lat
    long = point.long
    # Find the nearest edge to the location point
    nearest_edge = ox.distance.nearest_edges(graph_of_stops, X=long, Y=lat, return_dist=False)
    node1, node2, key = nearest_edge
    edge = graph_of_stops.get_edge_data(node1, node2)
    data = list(edge.values())[0]
        
    # get the line
    if 'geometry' not in data:
        gdf_edges = ox.graph_to_gdfs(graph_of_stops, nodes=False, edges=True)
        line = gdf_edges.loc[nearest_edge]['geometry']
    else:
        line = data['geometry']
        
    point_a = (graph_of_stops.nodes[node1]['y'], graph_of_stops.nodes[node1]['x'])
    point_c = (graph_of_stops.nodes[node2]['y'], graph_of_stops.nodes[node2]['x']) 
    

    graph_of_stops.add_node(point.id, x=long, y=lat, isTranspo=is_transpo)
    list_of_stops.append(point)

    # Get the points
    point_b = (lat, long) # y,x
    
    # Calculate the new distances
    distance_ab = geodesic(point_a, point_b).meters
    distance_bc = geodesic(point_b, point_c).meters
        
    # Get the edge data and adjust distances
    edge_data = data.copy()
    edge_data['length'] = distance_ab
    graph_of_stops.add_edge(node1, point.id, **edge_data)
    
    edge_data['length'] = distance_bc
    graph_of_stops.add_edge(point.id, node2, **edge_data)

    # Remove the original edge
    graph_of_stops.remove_edge(node1, node2)
        
def calculate_coordinate_along_edge(edge, position):
    # Calculate the coordinate along the edge at the given position
    point = edge.interpolate(position)
    return point.x, point.y

In [14]:
# DELETE AFTER
# Checks if point is out of the specific geographic location
def remove_point_out_of_bounds(city_graph, route, tolerance):
    nodes = ox.graph_to_gdfs(city_graph, edges=False)
    min_x, min_y, max_x, max_y = nodes.total_bounds
    
    new_route = []
    for point in route:
        lat = point[0]
        long = point[1]
        
        if min_x <= long <= max_x and min_y <= lat <= max_y:
            # Convert the graph to GeoDataFrame for easier spatial operations
            nearest_edge, distance_to_edge = ox.distance.nearest_edges(city_graph, X=long, Y=lat, return_dist=True)
            
        
            # Return True if any distance is less than the tolerance
            if distance_to_edge < tolerance and [lat, long] not in manila_excluded_stops:
                new_route.append(point)
            else:
                break

    return new_route

In [15]:
import re

def process_csv(file_path):
    with open(file_path, mode='r', newline='') as file:
        point_pattern = re.compile(r'POINT \(([-\d.]+) ([-\d.]+)\)')
        reader = csv.DictReader(file)
        
        # Variables to hold the current route list and all routes list
        current_route_number = None
        current_route_list = []
        all_routes_list = []

        for row in reader:
            wkt = row['WKT']
            route_number = row['Route Number']
            
            match = point_pattern.match(wkt)
            if match:
                x, y = match.groups()
                x, y = float(x), float(y)
            else:
                x, y = None, None

            # Check if the route number has changed
            if route_number != current_route_number:
                # If there's a current route list, append it to the all routes list
                if current_route_list:
                    all_routes_list.append(current_route_list)
                
                # Start a new route list
                current_route_list = []
                current_route_number = route_number

            # Append the current row to the current route list
            current_route_list.append([(x, y), route_number])

        # Append the last route list to the all routes list
        if current_route_list:
            all_routes_list.append(current_route_list)
        
    return all_routes_list

### Run

In [ ]:
# MANDALUYONG
file_path = 'Existing Jeepney Routes (Mandaluyong)'
routes = process_csv(file_path)
for route in routes:
    for point in route:
        print(point)

In [18]:
map_center = (14.599512, 120.984222)
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

# Plotting in the Map
for stop in routes[0]:
        lat = stop[0][1]
        long = stop[0][0]
        folium.Marker(location=[lat, long], popup=f"{lat}, {long}", icon=folium.Icon(color='blue')).add_to(m)
    
# for connection in test_routes:
#         ox.plot_route_folium(test_city_graph, connection, route_map=m, tiles='openstreetmap', route_color="green")

m.save(f"test_map.html")
webbrowser.open("test_map.html")

True

In [27]:
test_city_graph = MANILA_GRAPH.copy()

In [26]:
# DELETE AFTER - Change to stopcandidate
list_stops = []
i = 0
for stop in routes[0]:
    lat = stop[0][1]
    long = stop[0][0]
    list_stops.append(stopCandidate(lat, long, False, i))
    i += 1

In [36]:
list_of_stops_test = []
for stop in list_stops:
    add_point_to_graph(stop, test_city_graph, list_of_stops_test, False)

# Adding to graph
test_routes = []
prev_node = None
for stop in list_stops:
    if prev_node == None:
        prev_node = stop
        continue
    
    path = nx.shortest_path(test_city_graph, prev_node.id, stop.id)
    print(f"{prev_node.id} - {stop.id}")
    test_routes.append(path)
    prev_node = stop
# Map


# ROUTE NETWORK MAP
map_center = (14.599512, 120.984222)
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

# Plotting in the Map
for stop in routes[0]:
    lat = stop[0][1]
    long = stop[0][0]
    folium.Marker(location=[lat, long], popup=f"{lat}, {long}", icon=folium.Icon(color='blue')).add_to(m)

for connection in test_routes:
        ox.plot_route_folium(test_city_graph, connection, route_map=m, tiles='openstreetmap', route_color="green")

m.save(f"test_map_with_roads.html")
webbrowser.open("test_map.html")

0 - 1
1 - 2
2 - 3
3 - 4
4 - 5
5 - 6
6 - 7
7 - 8
8 - 9
9 - 10
10 - 11
11 - 12
12 - 13
13 - 14
14 - 15
15 - 16
16 - 17
17 - 18
18 - 19
19 - 20
20 - 21
21 - 22
22 - 23
23 - 24
24 - 25
25 - 26
26 - 27
27 - 28


/var/folders/q0/zbq8f0v11vb5hv6jh21spfwm0000gn/T/ipykernel_17006/896813368.py:31: FutureWarning: The `folium` module has been deprecated and will be removed in the v2.0.0 release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  ox.plot_route_folium(test_city_graph, connection, route_map=m, tiles='openstreetmap', route_color="green")
/var/folders/q0/zbq8f0v11vb5hv6jh21spfwm0000gn/T/ipykernel_17006/896813368.py:31: FutureWarning: The `folium` module has been deprecated and will be removed in the v2.0.0 release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for co

True